In [1]:
import sys
sys.path.append("C:/Users/user/meepc")
import numpy as np
import pandas as pd
from models import Hankel,Corrhankel,Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def calculate_fpr(y_actual, y_prediction):
    fp = 0
    tn = 0
    
    for i in range(len(y_actual)):
        if y_actual[i] == 0 and y_prediction[i] == 1:
            fp += 1
        elif y_actual[i] == 0 and y_prediction[i] == 0:
            tn += 1
    fpr= fp/(fp+tn)
    return fpr

In [3]:
train_normal = pd.read_csv('~/data/te/normal_training.csv')
train_attack = pd.read_csv('~/data/te/attack_training.csv')

In [4]:
sensors = [col for col in train_normal.columns if col not in ['faultNumber', 'simulationRun', 'sample','LABEL']]

In [5]:
scaler = StandardScaler()
X_normal = pd.DataFrame(index=train_normal.index, columns=sensors, data=scaler.fit_transform(train_normal[sensors]))
X_attack = pd.DataFrame(data=scaler.fit_transform(train_attack.loc[:,sensors]),index=train_attack.index,columns=sensors).reset_index().drop(columns=['index'])

In [6]:
hankel = Hankel()
corrhankel = Corrhankel()
lag = 500
stride = 0.5

In [7]:
corr_normal,nolag_normal = corrhankel.fit(X_normal.to_numpy(),lag,stride)

In [8]:
corr_attack,nolag_attack = corrhankel.fit(X_attack.to_numpy(),lag,stride)

In [9]:
df_test1 = pd.read_csv('~/data/te/normal_testing.csv')
df_test1 = df_test1.sample(n=15000, random_state=42)
df_test1.reset_index(drop=True, inplace=True)
df_test2 = pd.read_csv('~/data/te/attack_testing.csv')
df_test2 = df_test2.groupby('faultNumber').apply(lambda x: x.sample(n=150, random_state=42))
df_test2.reset_index(drop=True, inplace=True)
df_test=pd.concat((df_test1,df_test2),axis=0)

# Epasad with 1 cluster and no threshold tuning (training attack included in test data)

In [10]:
test_combined = pd.concat((df_test,train_attack),axis=0)
X_test = pd.DataFrame(index=test_combined.index, columns=sensors, data=scaler.fit_transform(test_combined[sensors]))
corr_test,nolag_test = corrhankel.fit(X_test.to_numpy(),lag,stride)
Y_test = test_combined.loc[:,'LABEL']

In [11]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [12]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
fpr = []
for i,sens in enumerate(sensors):
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,only_corr=True)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test,corr_test[:,i].reshape(nolag_test,len(X_test.columns)).T)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    fpr.append(calculate_fpr(y_actual,y_predicted))
    sensor_models.append(model)    

In [25]:
sensor_predicted = np.asarray(sensor_predicted)
sensor_mean = np.mean(sensor_predicted, axis=0)
y_predicted = (sensor_mean > 0.75).astype(int)
# y_predicted = np.any(sensor_predicted,axis=0).astype(int)
y_predicted

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0])

In [26]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))
print("False Positive Rate ",calculate_fpr(y_actual,y_predicted))

Accuracy  0.8545454545454545
Precision  1.0
Recall  0.6862745098039216
F1-score  0.813953488372093
False Positive Rate  0.0


In [27]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.579020979020979 Median : 0.4636363636363636 Min : 0.4 Max : 0.9363636363636364
precision - Mean: 0.629502357409023 Median : 0.4636363636363636 Min : 0.42718446601941745 Max : 1.0
recall -    Mean: 0.8563348416289591 Median : 1.0 Min : 0.37254901960784315 Max : 1.0
f1 -        Mean: 0.6609563374962986 Median : 0.6335403726708074 Min : 0.5428571428571428 Max : 0.9263157894736842


# Multiple clusters + No threshold tuning (training attack mixed in test data)

In [29]:
cluster_accuracy_1 = []
cluster_precision_1 = []
cluster_recall_1 = []
cluster_fscore_1 = []
cluster_fpr_1 = []
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    fpr = []
    for i,sens in enumerate(sensors):
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k = k,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,only_corr=True)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_test.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        fpr.append(calculate_fpr(y_actual,y_predicted))
        sensor_models.append(model)  
    sensor_predicted = np.asarray(sensor_predicted)
    sensor_mean = np.mean(sensor_predicted, axis=0)
    y_predicted = (sensor_mean > 0.75).astype(int)
    # y_predicted = np.any(sensor_predicted,axis=0).astype(int) 
    cluster_accuracy_1.append(accuracy_score(y_actual, y_predicted))
    cluster_precision_1.append(precision_score(y_actual,y_predicted))
    cluster_recall_1.append(recall_score(y_actual,y_predicted))
    cluster_fscore_1.append(f1_score(y_actual,y_predicted))
    cluster_fpr_1.append(calculate_fpr(y_actual,y_predicted))
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",cluster_accuracy_1[-1])
    print("Precision ",cluster_precision_1[-1])
    print("Recall ",cluster_recall_1[-1])
    print("F1-score ",cluster_fscore_1[-1])
    print("False Positive Rate ",cluster_fpr_1[-1],"\n") 

------Number of Clusters:  2 -----------
Accuracy  0.8727272727272727
Precision  0.9743589743589743
Recall  0.7450980392156863
F1-score  0.8444444444444443
False Positive Rate  0.01694915254237288 

------Number of Clusters:  3 -----------
Accuracy  0.8909090909090909
Precision  0.9534883720930233
Recall  0.803921568627451
F1-score  0.8723404255319148
False Positive Rate  0.03389830508474576 

------Number of Clusters:  4 -----------
Accuracy  0.8909090909090909
Precision  1.0
Recall  0.7647058823529411
F1-score  0.8666666666666666
False Positive Rate  0.0 

------Number of Clusters:  5 -----------
Accuracy  0.8363636363636363
Precision  0.8367346938775511
Recall  0.803921568627451
F1-score  0.8200000000000001
False Positive Rate  0.13559322033898305 

------Number of Clusters:  6 -----------
Accuracy  0.8272727272727273
Precision  0.8478260869565217
Recall  0.7647058823529411
F1-score  0.8041237113402062
False Positive Rate  0.11864406779661017 

------Number of Clusters:  7 ---------

In [30]:
print("Accuracy Scores: ",cluster_accuracy_1)
print("Precision Scores: ",cluster_precision_1)
print("Recall Scores: ",cluster_recall_1)
print("F1 Scores: ",cluster_fscore_1)
print("False Positive Rates: ",cluster_fpr_1)

Accuracy Scores:  [0.8727272727272727, 0.8909090909090909, 0.8909090909090909, 0.8363636363636363, 0.8272727272727273, 0.8181818181818182]
Precision Scores:  [0.9743589743589743, 0.9534883720930233, 1.0, 0.8367346938775511, 0.8478260869565217, 0.8444444444444444]
Recall Scores:  [0.7450980392156863, 0.803921568627451, 0.7647058823529411, 0.803921568627451, 0.7647058823529411, 0.7450980392156863]
F1 Scores:  [0.8444444444444443, 0.8723404255319148, 0.8666666666666666, 0.8200000000000001, 0.8041237113402062, 0.7916666666666666]
False Positive Rates:  [0.01694915254237288, 0.03389830508474576, 0.0, 0.13559322033898305, 0.11864406779661017, 0.11864406779661017]


# Multiple clusters + No threshold tuning (No concat of training and test data)

In [31]:
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
corr_test,nolag_test = corrhankel.fit(X_test.to_numpy(),lag,stride)
Y_test = df_test.loc[:,'LABEL']

In [32]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [34]:
cluster_accuracy_2 = []
cluster_precision_2 = []
cluster_recall_2 = []
cluster_fscore_2 = []
cluster_fpr_2 = []
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    fpr = []
    for sens in sensors:
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,only_corr=True)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        fpr.append(calculate_fpr(y_actual,y_predicted))
        sensor_models.append(model)    
    sensor_predicted = np.asarray(sensor_predicted)
    sensor_mean = np.mean(sensor_predicted, axis=0)
    y_predicted = (sensor_mean > 0.75).astype(int)
    # y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    cluster_accuracy_2.append(accuracy_score(y_actual, y_predicted))
    cluster_precision_2.append(precision_score(y_actual,y_predicted))
    cluster_recall_2.append(recall_score(y_actual,y_predicted))
    cluster_fscore_2.append(f1_score(y_actual,y_predicted))
    cluster_fpr_2.append(calculate_fpr(y_actual,y_predicted))
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",cluster_accuracy_2[-1])
    print("Precision ",cluster_precision_2[-1])
    print("Recall ",cluster_recall_2[-1])
    print("F1-score ",cluster_fscore_2[-1])
    print("False Positive Rate ",cluster_fpr_2[-1],"\n") 

------Number of Clusters:  2 -----------
Accuracy  0.9857142857142858
Precision  0.9166666666666666
Recall  1.0
F1-score  0.9565217391304348
False Positive Rate  0.01694915254237288 

------Number of Clusters:  3 -----------
Accuracy  0.9714285714285714
Precision  0.8461538461538461
Recall  1.0
F1-score  0.9166666666666666
False Positive Rate  0.03389830508474576 

------Number of Clusters:  4 -----------
Accuracy  0.9285714285714286
Precision  0.6875
Recall  1.0
F1-score  0.8148148148148148
False Positive Rate  0.0847457627118644 

------Number of Clusters:  5 -----------
Accuracy  0.9142857142857143
Precision  0.6470588235294118
Recall  1.0
F1-score  0.7857142857142858
False Positive Rate  0.1016949152542373 

------Number of Clusters:  6 -----------
Accuracy  0.8714285714285714
Precision  0.55
Recall  1.0
F1-score  0.7096774193548387
False Positive Rate  0.15254237288135594 

------Number of Clusters:  7 -----------
Accuracy  0.8571428571428571
Precision  0.5238095238095238
Recall  

In [35]:
print("Accuracy Scores: ",cluster_accuracy_2)
print("Precision Scores: ",cluster_precision_2)
print("Recall Scores: ",cluster_recall_2)
print("F1 Scores: ",cluster_fscore_2)
print("False Positive Rates: ",cluster_fpr_2)

Accuracy Scores:  [0.9857142857142858, 0.9714285714285714, 0.9285714285714286, 0.9142857142857143, 0.8714285714285714, 0.8571428571428571]
Precision Scores:  [0.9166666666666666, 0.8461538461538461, 0.6875, 0.6470588235294118, 0.55, 0.5238095238095238]
Recall Scores:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
F1 Scores:  [0.9565217391304348, 0.9166666666666666, 0.8148148148148148, 0.7857142857142858, 0.7096774193548387, 0.6875000000000001]
False Positive Rates:  [0.01694915254237288, 0.03389830508474576, 0.0847457627118644, 0.1016949152542373, 0.15254237288135594, 0.1694915254237288]


# Multiple clusters + Threshold tuning (No concat of training and test data)

In [37]:
cluster_accuracy_3 = []
cluster_precision_3 = []
cluster_recall_3 = []
cluster_fscore_3 = []
cluster_fpr_3 = []
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    fpr = []
    for sens in sensors:
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k = k,tune=True,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,corr_attack=corr_attack[:,i].reshape(nolag_attack,len(X_attack.columns)).T,only_corr=True)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        fpr.append(calculate_fpr(y_actual,y_predicted))
        sensor_models.append(model)    
    sensor_predicted = np.asarray(sensor_predicted)
    sensor_mean = np.mean(sensor_predicted, axis=0)
    y_predicted = (sensor_mean > 0.75).astype(int)
    # y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    cluster_accuracy_3.append(accuracy_score(y_actual, y_predicted))
    cluster_precision_3.append(precision_score(y_actual,y_predicted))
    cluster_recall_3.append(recall_score(y_actual,y_predicted))
    cluster_fscore_3.append(f1_score(y_actual,y_predicted))
    cluster_fpr_3.append(calculate_fpr(y_actual,y_predicted))
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",cluster_accuracy_3[-1])
    print("Precision ",cluster_precision_3[-1])
    print("Recall ",cluster_recall_3[-1])
    print("F1-score ",cluster_fscore_3[-1])
    print("False Positive Rate ",cluster_fpr_3[-1],"\n")

------Number of Clusters:  2 -----------
Accuracy  0.9857142857142858
Precision  0.9166666666666666
Recall  1.0
F1-score  0.9565217391304348
False Positive Rate  0.01694915254237288 

------Number of Clusters:  3 -----------
Accuracy  0.9714285714285714
Precision  0.8461538461538461
Recall  1.0
F1-score  0.9166666666666666
False Positive Rate  0.03389830508474576 

------Number of Clusters:  4 -----------
Accuracy  0.9571428571428572
Precision  0.7857142857142857
Recall  1.0
F1-score  0.88
False Positive Rate  0.05084745762711865 

------Number of Clusters:  5 -----------
Accuracy  0.9142857142857143
Precision  0.6470588235294118
Recall  1.0
F1-score  0.7857142857142858
False Positive Rate  0.1016949152542373 

------Number of Clusters:  6 -----------
Accuracy  0.8857142857142857
Precision  0.5789473684210527
Recall  1.0
F1-score  0.7333333333333334
False Positive Rate  0.13559322033898305 

------Number of Clusters:  7 -----------
Accuracy  0.8857142857142857
Precision  0.578947368421

In [38]:
print("Accuracy Scores: ",cluster_accuracy_3)
print("Precision Scores: ",cluster_precision_3)
print("Recall Scores: ",cluster_recall_3)
print("F1 Scores: ",cluster_fscore_3)
print("False Positive Rates: ",cluster_fpr_3)

Accuracy Scores:  [0.9857142857142858, 0.9714285714285714, 0.9571428571428572, 0.9142857142857143, 0.8857142857142857, 0.8857142857142857]
Precision Scores:  [0.9166666666666666, 0.8461538461538461, 0.7857142857142857, 0.6470588235294118, 0.5789473684210527, 0.5789473684210527]
Recall Scores:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
F1 Scores:  [0.9565217391304348, 0.9166666666666666, 0.88, 0.7857142857142858, 0.7333333333333334, 0.7333333333333334]
False Positive Rates:  [0.01694915254237288, 0.03389830508474576, 0.05084745762711865, 0.1016949152542373, 0.13559322033898305, 0.13559322033898305]
